# SPARK STREAMING

Desde el SHELL del cluster crearemos el directorio stream y dos ficheros de texto

hdfs dfs -mkdir /stream

nano text1

el caso del perro verde

nano text2

verde verde que te quiero verde

Generaremos un contexto de Spark Streaming y utilizaremos le método textFileStream para crear un stream que lea ficheros de texto tan pronto como lleguen a la carpeta /stream. Los datos se cargan en un RDD, y el texto se divide en palabras, que se cuentan dentro de una función sliding window que incluye los últimos 60 segundos de datos a intervalos de 10 segundos. El método pprint se utiliza para escribir los resultados de cada batch en consola

In [ ]:
from pyspark.streaming import StreamingContext
# Crear un  StreamingContext
ssc = StreamingContext(sc, 1)
ssc.checkpoint("wasb:///chkpnt")
# Definir un stream de fichero de texto para la carpeta /stream
streamRdd = ssc.textFileStream("wasb:///stream")
# contar las palabras
words = streamRdd.flatMap(lambda line: line.split(" "))
pairs = words.map(lambda word: (word, 1))
wordCounts = pairs.reduceByKeyAndWindow(lambda a, b: a + b, lambda x, y: x - y, 60, 10)
# ver los primeros 20 elementos del DStream
wordCounts.pprint(num=20)
ssc.start()

Desde el shell vamos generando los ficheros en la carpeta

hdfs dfs -put text1 /stream/text1_1

esperar unos pocos segundos

hdfs dfs -put text1 /stream/text1_2

esperar unos pocos segundos

hdfs dfs -put text2 /stream/text2_1

continuar subiendo ficheros....


In [ ]:
#Paramos la aplicaicón de stream
ssc.stop()
